In [111]:
import pandas as pd
import random

In [112]:
dye_names = [ "8E","P","M","D","H"]
work_days = 5
capacity_per_day = {
"8E":288000,
"P":57600,
"M":100800,
"D":72000,
"H":86400,
}

In [113]:
def random_requirement_generator(size: int) -> dict:
    for i in range(size):
        yield {
            "part_id": i+1,
            "dye":random.choice(dye_names),
            "cust": random.choice(["Autoliv","KSS","TRW"]),
            "safety_stock":0,
            "cards":random.randint(0,5),
            "cards_quantity":random.randint(0,14401),
        }
        
requirments = list(req for req in random_requirement_generator(100))

In [114]:
requirements_df = pd.DataFrame.from_records(data=requirments)

requirements_df["build_quantity"] = list(
    row["cards"]*row["cards_quantity"] for index, row in requirements_df.iterrows() 
)
requirements_df["dye_capacity"] = list (
    capacity_per_day[row["dye"]]*work_days for index, row in requirements_df.iterrows()
)

total_build_quantity = requirements_df.groupby(by=requirements_df["dye"]) \
                        .sum() \
                        .reset_index() \
                        .drop(
                            columns=["part_id","safety_stock","cards","cards_quantity","dye_capacity"]
                        ) \
                        .rename({
                            "build_quantity":"total_build_qty"
                        })


In [115]:
total_build_quantity

,dye,build_quantity
0,8E,427409
1,D,323518
2,H,394015
3,M,486509
4,P,243040


In [116]:
requirements_df

,part_id,dye,cust,safety_stock,cards,cards_quantity,build_quantity,dye_capacity
0,1,H,TRW,0,4,492,1968,432000
1,2,8E,KSS,0,5,5301,26505,1440000
2,3,M,TRW,0,4,9921,39684,504000
3,4,8E,Autoliv,0,4,9045,36180,1440000
4,5,M,TRW,0,4,12432,49728,504000
...,...,...,...,...,...,...,...,...
95,96,P,Autoliv,0,3,10520,31560,288000
96,97,8E,Autoliv,0,4,8070,32280,1440000
97,98,D,Autoliv,0,1,5857,5857,360000
98,99,M,Autoliv,0,4,12457,49828,504000
